In [ ]:
#댓글 유저를 특정해서 댓글을 단 기사 crawling
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import numpy as np
import time
from datetime import datetime
import uuid  # To generate unique identifiers

# 시간으로 크롤링 시간 제한
time_limit = 1200  # 20 minutes, change to 3600 for 1 hour

start_time = time.time()

# 크롬 드라이버 작동
driver = webdriver.Chrome()
driver.get('https://n.news.naver.com/mnews/article/comment/243/0000058980#user_comment_824138786735652991_news243')
time.sleep(3)

# date, time과 unique identifier를 구분하여 저장
def save_info_to_file(comment, date, source, link_href):
    # Use the date element to create a file name
    date_for_filename = date.split()[0].replace('.', '-')  # Format date for file name
    time_for_filename = datetime.now().strftime("%H_%M_%S")  # Get current time for unique filename
    unique_id = uuid.uuid4().hex[:6]  # Generate a unique ID
    file_name = f"C:\\Users\\rapha\\PJ\\PJ_04\\commentor\\{date_for_filename}_{time_for_filename}_{unique_id}_comment_data.txt"

    
    # text 파일로 저장
    with open(file_name, 'w', encoding='utf-8') as f:
        f.write(f"Comment: {comment}\n")
        f.write(f"Date: {date}\n")
        f.write(f"Source: {source}\n")
        f.write(f"Link: {link_href}\n")
        f.write("-" * 50 + "\n")  # Separator
    #print(f"Info saved to file: {file_name}")

# 시간제한까지 반복실행
while True:
    # Check if the time limit has been reached
    if time.time() - start_time > time_limit:
        print("Time limit reached. Stopping the script.")
        break

    try:
        # "더 보기" 버튼 누르기
        more_button = driver.find_element(By.CSS_SELECTOR, '#wa_allcomments > div.u_cbox_paginate.is_more_button > a > span > span > span.u_cbox_page_more')
        more_button.click()
        time.sleep(2)
    except Exception as e:
        print("No more comments to load or error occurred:", e)
        break

    # Parsing
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 댓글 부분 선택택
    comment_elements = soup.select('#cbox_module_wai_u_cbox_content_wrap_tabpanel > ul > li.u_cbox_comment')

    for comment_element in comment_elements:
        # 링크, 삭제된 본문, 날짜 정보 추출
        link_element = comment_element.select_one('div.u_cbox_comment_box.u_cbox_type_profile > div > div.u_cbox_text_wrap > span.u_cbox_contents > a')
        deleted_text_element = comment_element.select_one('div.u_cbox_orgsourcedel > span.u_cbox_orgsourcedel_txt')
        orgsource_element = comment_element.select_one('div.u_cbox_comment_box.u_cbox_type_profile > div > div.u_cbox_orgsource > a > div.u_cbox_orgsource_content > strong')
        link_element_2 = comment_element.select_one('div.u_cbox_comment_box.u_cbox_type_profile > div > div.u_cbox_orgsource > a')
        date_element = comment_element.select_one('div.u_cbox_comment_box.u_cbox_type_profile > div > div.u_cbox_info_base > span')

        # 데이터 저장 목록
        comment_text = "N/A"
        date_text = "N/A"
        source_text = "N/A"
        link_href = "N/A"

        # 데이터 수집
        if link_element:
            comment_text = link_element.get_text()
            print("Comment:", comment_text)

        if date_element:
            date_text = date_element.get_text()
            print("Date:", date_text)

        if orgsource_element:
            source_text = orgsource_element.get_text()
            print("Source:", source_text)

        if deleted_text_element and deleted_text_element.get_text() == '언론사에 의해 삭제된 기사입니다.':
            source_text = '언론사에 의해 삭제된 기사입니다.'
            print("Deleted article detected.")

        if link_element_2:
            link_href = link_element_2.get('href')
            print("Link href:", link_href)

        # 수집된 데이터 저장
        save_info_to_file(comment_text, date_text, source_text, link_href)
        
driver.quit()

In [5]:
#text 파일 합치기(na 제외)
import os

directory = r"C:\Users\rapha\PJ\PJ_04\commentor"

output_file = "concatenated_comments.txt"

# 'N/A'값이 있는지 확인
def contains_na(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
        return 'N/A' in content

# 저장 파일 
with open(output_file, 'w', encoding='utf-8') as outfile:
    # Iterate through all files in the directory
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        
        # 'N/A'값이 있는지 확인
        if not contains_na(file_path):
            # If not, read the file and write its content to the output file
            with open(file_path, 'r', encoding='utf-8') as infile:
                outfile.write(infile.read())
                outfile.write("\n" + "-" * 50 + "\n")  # Separator between files

print(f"Concatenated data saved to {output_file}")

Concatenated data saved to concatenated_comments.txt


In [8]:
# csv로 전환
import csv

input_file = r"C:\Users\rapha\PJ\PJ_04\concatenated_comments.txt"

output_csv = "comments_data.csv"

data = []

# 텍스트파일 불러오기
with open(input_file, 'r', encoding='utf-8') as infile:
    lines = infile.readlines()
    
    comment = date = source = link = None
    
    # 데이터 추출
    for line in lines:
        if line.startswith("Comment:"):
            comment = line.split("Comment:")[1].strip()
        elif line.startswith("Date:"):
            date = line.split("Date:")[1].strip()
        elif line.startswith("Source:"):
            source = line.split("Source:")[1].strip()
        elif line.startswith("Link:"):
            link = line.split("Link:")[1].strip()
        elif line.startswith("-" * 50):
            # 추출된 데이터 리스트에 추가
            if comment and date and source and link:
                data.append([comment, date, source, link])
            # 다음 댓글을 위해 초기화
            comment = date = source = link = None

with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["Comment", "Date", "Source", "Link"])
    csvwriter.writerows(data)

print(f"Data successfully saved to {output_csv}")

Data successfully saved to comments_data.csv
